In [49]:
df = pd.read_csv('Landsat_Flooded_Areas_Z2C.csv')

df = df.drop(columns=['system:index', '.geo'])

df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

df = df.set_index('date')

max_area = df['totalarea'].max()

threshold_area = max_area*0.85

df = df[(df['totalarea']>threshold_area)]

df['flooded'] = df['flood']/df['totalarea']

df = df.drop(columns=['dry', 'flood', 'totalarea'])

Zone2C  = df

In [50]:
df = pd.read_csv('Landsat_Flooded_Areas_PA.csv')

df = df.drop(columns=['system:index', '.geo'])

df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

df = df.set_index('date')

max_area = df['totalarea'].max()

threshold_area = max_area*0.85

df = df[(df['totalarea']>threshold_area)]

df['flooded'] = df['flood']/df['totalarea']

df = df.drop(columns=['dry', 'flood', 'totalarea'])

PA  = df

In [51]:
# water level data at Koh Kehl 
wl_unsmoothed = pd.read_csv('wl_KK.csv')
wl_unsmoothed['date'] = pd.to_datetime(wl_unsmoothed['date'])
wl_unsmoothed = wl_unsmoothed.set_index('date')
wl_unsmoothed = wl_unsmoothed.resample('D').max()
wl_unsmoothed = wl_unsmoothed.drop(columns=['Unnamed: 0'])
wl_unsmoothed.columns = ['WL']
wl = wl_unsmoothed.rolling(5, min_periods=5).mean()
wl['date'] = wl.index
wl['year'] = wl['date'].dt.strftime("%Y")
wl['WL'] = wl['WL'].astype(float)
wl['WL'] = wl['WL']-1 #<------------ correct for station elevation



wl_KK = wl
wl_KK.head()

,WL,date,year
date,,,
1990-09-05,NaN,1990-09-05,1990
1990-09-06,NaN,1990-09-06,1990
1990-09-07,NaN,1990-09-07,1990
1990-09-08,NaN,1990-09-08,1990
1990-09-09,7.202,1990-09-09,1990


In [52]:
merged_PA = PA.merge(wl_KK, how="right", left_index=True, right_index=True)

merged_PA.to_csv('merged_PA_thresh.csv')


merged_Z2C = Zone2C.merge(wl_KK, how="right", left_index=True, right_index=True)

merged_Z2C.to_csv('merged_Z2C_thresh.csv')

In [53]:

count_PA.head()

,flooded,WL,date,year,count_reached
1,0.493423,0.578,2020-05-20,2020,1
2,0.470522,0.698,2020-06-05,2020,2
3,0.609507,0.726,1998-05-24,1998,3
4,0.461384,0.820,2020-06-21,2020,4
5,0.465081,1.008,2019-06-03,2019,5


In [54]:
count_PA = PA.merge(wl_KK, how="inner", left_index=True, right_index=True)

count_PA = count_PA.sort_values('WL')

count_PA.index = np.arange(1, len(count_PA) + 1)

count_PA['count_reached'] = count_PA.index

count_PA.to_csv('count_PA_thresh.csv')



####------------------------------------


count_Z2C = Zone2C.merge(wl_KK, how="inner", left_index=True, right_index=True)

count_Z2C = count_Z2C.sort_values('WL')

count_Z2C.index = np.arange(1, len(count_Z2C) + 1)

count_Z2C['count_reached'] = count_Z2C.index

count_Z2C.to_csv('count_Z2C_thresh.csv')